In [129]:
import pandas as pd
import numpy as np

from pandas import Series,DataFrame
from sklearn.linear_model import LogisticRegression

In [130]:
bank_full = pd.read_csv('bank_full_w_dummy_vars.csv')

In [131]:
bank_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,job_unknown,job_retired,job_services,job_self_employed,job_unemployed,job_maid,job_student,married,single,divorced
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,...,0,0,0,0,0,0,0,1,0,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,...,0,0,0,0,0,0,0,0,1,1
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,...,0,0,0,0,0,0,0,1,0,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,...,0,0,0,0,0,0,0,1,0,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,...,1,0,0,0,0,0,0,0,1,1


In [132]:
bank_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 37 columns):
age                             45211 non-null int64
job                             45211 non-null object
marital                         45211 non-null object
education                       45211 non-null object
default                         45211 non-null object
balance                         45211 non-null int64
housing                         45211 non-null object
loan                            45211 non-null object
contact                         45211 non-null object
day                             45211 non-null int64
month                           45211 non-null object
duration                        45211 non-null int64
campaign                        45211 non-null int64
pdays                           45211 non-null int64
previous                        45211 non-null int64
poutcome                        45211 non-null object
y                               45

In [133]:
X_bank = bank_full.ix[:,(18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36)].values

y = bank_full.ix[:,17].values

C:\Users\Shweta\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\Shweta\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [134]:
LogReg = LogisticRegression()
LogReg.fit(X_bank,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [135]:
new_user = [[0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1]]
y_pred = LogReg.predict(new_user)

In [136]:
y_pred

array([0], dtype=int64)

### So the conclusion is user will not accept the offer given to him

# Model based collaborative filtering


# SVD

In [137]:
from sklearn.decomposition import TruncatedSVD

In [138]:
columns_user = ['user_id','item_id','rating','timetstamp']
frame = pd.read_csv('u.data',sep='\t',names=columns_user)

In [139]:
frame.head()

,user_id,item_id,rating,timetstamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [140]:
columns_movies = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('u.item',sep='|',names=columns_movies,encoding = 'latin-1')

movies = movies[['item_id', 'movie title']]

In [141]:
movies.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [142]:
combined_movies_Data = pd.merge(frame,movies,on='item_id')

In [143]:
combined_movies_Data.head()

,user_id,item_id,rating,timetstamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [144]:
combined_movies_Data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [145]:
Filter = combined_movies_Data['item_id'] == 50
combined_movies_Data[Filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

#### Utility matrix

In [146]:
rating_crosstab = combined_movies_Data.pivot_table(values='rating',index='user_id',columns='movie title',fill_value=0)

In [147]:
rating_crosstab.head()

movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


#### Transpose the matrix

In [148]:
rating_crosstab.shape

(943, 1664)

In [149]:
X_movies = rating_crosstab.values.T

In [150]:
X_movies.shape

(1664, 943)

#### Decompose

In [151]:
SVD = TruncatedSVD(n_components=12,random_state=17)

In [153]:
resultant_matrix= SVD.fit_transform(X_movies)

resultant_matrix.shape

(1664, 12)

#### Correlation matrix

In [154]:
corr_mat = np.corrcoef(resultant_matrix)

In [155]:
corr_mat.shape

(1664, 1664)

In [156]:
movies_names = rating_crosstab.columns

In [157]:
movies_list = list(movies_names)

In [158]:
star_wars = movies_list.index('Star Wars (1977)')

In [159]:
corr_star_wars = corr_mat[star_wars]

In [160]:
corr_star_wars.shape

(1664,)

#### Recommend highly correlated movie

In [161]:
list(movies_names[(corr_star_wars < 1.0) & (corr_star_wars>0.9)])

['Die Hard (1988)',
 'Empire Strikes Back, The (1980)',
 'Fugitive, The (1993)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Star Wars (1977)',
 'Terminator 2: Judgment Day (1991)',
 'Terminator, The (1984)',
 'Toy Story (1995)']

In [162]:
list(movies_names[(corr_star_wars < 1.0) & (corr_star_wars>0.95)])

['Return of the Jedi (1983)', 'Star Wars (1977)']

# Content based recommender

In [163]:
from sklearn.neighbors import NearestNeighbors

In [164]:
cars = pd.read_csv('mtcars.csv')
cars.columns = ['car_names','mpg','cyl','disp','hp','drat','wt','qsec','vs','am','gear','carb']

In [165]:
cars.head()

,car_names,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [166]:
t = [10,300,160,3.2]

X_cars= cars.iloc[:,[1,3,4,6]].values

In [167]:
X_cars[0:5]

array([[ 21.   , 160.   , 110.   ,   2.62 ],
       [ 21.   , 160.   , 110.   ,   2.875],
       [ 22.8  , 108.   ,  93.   ,   2.32 ],
       [ 21.4  , 258.   , 110.   ,   3.215],
       [ 18.7  , 360.   , 175.   ,   3.44 ]])

In [168]:
nbrs = NearestNeighbors(n_neighbors=1).fit(X_cars)

In [169]:
nbrs.kneighbors([t])

(array([[11.96224164]]), array([[22]], dtype=int64))

In [170]:
cars.head(25)

,car_names,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,Duster 360,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,Merc 240D,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,Merc 230,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


### Conclusion : We should recommend car with name AMC Javelin which is present at index 22 in above cars dataset based on the user desires